In [ ]:
import geopandas
import os
from pygeotile.point import Point
from pygeotile.tile import Tile
from pygeotile.meta import TILE_SIZE
import requests
from PIL import Image
import random
from shutil import copy2
import json
import matplotlib.pyplot as plt

In [ ]:
import fiona

## Reading Polygons

In [ ]:
polygons_paths = [os.path.join('./', 'CityPolygon', 'polygon_edited.shp')]
lines_path = os.path.join('./', 'CityPolygon', 'polygon_lines_generated.shp')

## check zoom levels and resolution at: https://help.nearmap.com/kb/articles/293-zoom-and-x-y-values-in-tms
zoom = 18 # Zoom levels used to collect the multi-resolution images in the paper: 17, 18 and 19
res = 0.593 # 1.186, 0.593, and 0.297 for zoom levels 17, 18 and 19 respectively
data_name = "MELB-zoom18"
rgb_path = os.path.join('./', 'Gather_tiles', data_name, 'images')

In [ ]:
bf_gdf = 0
for i, polygons_path in enumerate(polygons_paths):
    if isinstance(bf_gdf, geopandas.geodataframe.GeoDataFrame):
        tmp_bf_gdf = geopandas.read_file(polygons_path)
        tmp_bf_gdf['file_id'] = i
        bf_gdf = bf_gdf.append(tmp_bf_gdf, ignore_index=True)
    else:
        bf_gdf = geopandas.read_file(polygons_path)
        bf_gdf['file_id'] = i

bf_gdf['bounds'] = bf_gdf.apply(lambda row: row['geometry'].bounds, axis=1)
bf_gdf['centroid'] = bf_gdf.apply(lambda row: row['geometry'].centroid, axis=1)
bf_gdf['i'] = bf_gdf.index
bf_gdf.head()

## Create Line features from polygons and save them

In [ ]:
bf_gdf_exploded = bf_gdf.explode()
bf_gdf_exploded_linestring = bf_gdf_exploded.set_geometry(bf_gdf_exploded.apply(lambda row: row['geometry'].boundary, axis = 1))
bf_gdf_exploded_linestring.reset_index()
bf_gdf_exploded_linestring.head()
bf_gdf_exploded_linestring.pop('bounds')
bf_gdf_exploded_linestring.pop('centroid')

bf_gdf_exploded_linestring.to_file(lines_path)

# Creating test and train data

## NearMap Tile API

In [ ]:
"""
You need an API key to acess the images:
https://api.nearmap.com/tiles/v3/{tileResourceType}/{z}/{x}/{y}.{format}?apikey={YOUR_API_KEY}
"""

existing_elements = set([])
employed_tiles = {}
for i, row in bf_gdf.iterrows():
    geom = row.geometry
    bound = row.bounds
    centroid = row.centroid
    file_id = row.file_id
    # if file_id==0:
    #     continue
    zoom = zoom  # meters in Spherical Mercator EPSG:900913
    point = Point.from_meters(centroid.x, centroid.y)
    print(point)
    tile = Tile.for_point(point, zoom)
    if (tile.google[0], tile.google[1]) not in existing_elements:
        path = "https://api.nearmap.com/tiles/v3/Vert/{zoom_level}/{x}/{y}.png"
        path = path.format(zoom_level=zoom,
                   x=tile.google[0],
                   y=tile.google[1])
        params = {
            'apikey': 'put your API key here'
        }
        response = requests.get(path, params=params)
        print(response)
        if response.status_code == 200:
            with open(os.path.join('Gather_tiles', data_name,'images', str(i)+'.png'), 'wb') as f:
                f.write(response.content)
            
            existing_elements.add((tile.google[0], tile.google[1]))
            employed_tiles[str(i)] = {'tile': (tile.google[0], tile.google[1], zoom), 'dataset_identifier': file_id}

In [ ]:
with open(os.path.join('Gather_tiles', data_name, 'employed_tiles.json'), 'w') as f:
    json.dump(employed_tiles, f)

## Making the mask/label dataset

In [ ]:
from autorasterize import rasterize

for k, obj in employed_tiles.items():
    tile = obj['tile']
    file_id = obj['dataset_identifier']
    google_x, google_y, zoom = tile # 119098, 82116, 17
    pixel_x_west, pixel_y_north = google_x * TILE_SIZE, google_y * TILE_SIZE
    pixel_x_east, pixel_y_south = (google_x + 1) * TILE_SIZE, (google_y + 1) * TILE_SIZE
    sw = Point.from_pixel(pixel_x_west, pixel_y_south, zoom).meters
    ne = Point.from_pixel(pixel_x_east, pixel_y_north, zoom).meters
    rasterize(polygons_paths[file_id], os.path.join('Gather_tiles', data_name,'label', str(k)+'.png') , sw[0], sw[1], ne[0], ne[1], 1.19)

## Test and Train Splitter

In [ ]:
rgb_files = [f for f in os.listdir(rgb_path) if os.path.isfile(os.path.join(rgb_path, f))]
random.shuffle(rgb_files)
files = {}
number_of_all_files = len(rgb_files)
number_of_train_files = int(number_of_all_files*0.7)
files['train'] = rgb_files[:number_of_train_files]
files['test'] = rgb_files[number_of_train_files:]

file_map_path = "./TrainingDataset/" + data_name + "/file_map.json"
with open(file_map_path, 'w') as f:
  print("dumping")
  json.dump(files, f)

In [ ]:
for k, list_of_files in files.items():
    counter = 0
    print("preparing ", k)
    out_path = os.path.join('./', 'TrainingDataset', data_name, k)
    for file in list_of_files:
        copy2(os.path.join(rgb_path, file), os.path.join(out_path, 'image', str(counter) + '.png'))
        copy2(os.path.join('Gather_tiles', data_name, 'label', file), os.path.join(out_path, 'label', str(counter) + '.png'))
        counter+=1

# All again for other zoom levels (17, 18, 19)

In [ ]:
""" Data and folder paths """
polygons_paths = [os.path.join('./', 'CityPolygon', 'polygon_edited.shp')]
lines_path = os.path.join('./', 'CityPolygon', 'polygon_lines_generated.shp')

## check zoom levels and resolution at: https://help.nearmap.com/kb/articles/293-zoom-and-x-y-values-in-tms
zoom = 18 # Zoom levels used to collect the multi-resolution images in the paper: 17, 18 and 19
res = 0.593 # 1.186, 0.593, and 0.297 for zoom levels 17, 18 and 19 respectively
data_name = "MELB-zoom18"
rgb_path = os.path.join('./', 'Gather_tiles', data_name, 'images')


""" Read data """
bf_gdf = 0
for i, polygons_path in enumerate(polygons_paths):
    if isinstance(bf_gdf, geopandas.geodataframe.GeoDataFrame):
        tmp_bf_gdf = geopandas.read_file(polygons_path)
        tmp_bf_gdf['file_id'] = i
        bf_gdf = bf_gdf.append(tmp_bf_gdf, ignore_index=True)
    else:
        bf_gdf = geopandas.read_file(polygons_path)
        bf_gdf['file_id'] = i

bf_gdf['bounds'] = bf_gdf.apply(lambda row: row['geometry'].bounds, axis=1)
bf_gdf['centroid'] = bf_gdf.apply(lambda row: row['geometry'].centroid, axis=1)
bf_gdf['i'] = bf_gdf.index
#bf_gdf.head()


""" Explode """
bf_gdf_exploded = bf_gdf.explode()
bf_gdf_exploded_linestring = bf_gdf_exploded.set_geometry(bf_gdf_exploded.apply(lambda row: row['geometry'].boundary, axis = 1))
bf_gdf_exploded_linestring.reset_index()
bf_gdf_exploded_linestring.head()
bf_gdf_exploded_linestring.pop('bounds')
bf_gdf_exploded_linestring.pop('centroid')

bf_gdf_exploded_linestring.to_file(lines_path)

"""
NEARMAP TILE API 
You need an API key to acess the images:
https://api.nearmap.com/tiles/v3/{tileResourceType}/{z}/{x}/{y}.{format}?apikey={YOUR_API_KEY}
"""
existing_elements = set([])
employed_tiles = {}
for i, row in bf_gdf.iterrows():
    geom = row.geometry
    bound = row.bounds
    centroid = row.centroid
    file_id = row.file_id
    # if file_id==0:
    #     continue
    zoom = zoom  # meters in Spherical Mercator EPSG:900913
    point = Point.from_meters(centroid.x, centroid.y)
    tile = Tile.for_point(point, zoom)
    
    if (tile.google[0], tile.google[1]) not in existing_elements:
        path = "https://api.nearmap.com/tiles/v3/Vert/{zoom_level}/{x}/{y}.png"
        path = path.format(zoom_level=zoom,
                   x=tile.google[0],
                   y=tile.google[1])
        params = {
            'apikey': 'put your API key here'
        }
        response = requests.get(path, params=params)
        print(response)
        if response.status_code == 200:
            with open(os.path.join('Gather_tiles', data_name,'images', str(i)+'.png'), 'wb') as f:
                f.write(response.content)
            
            existing_elements.add((tile.google[0], tile.google[1]))
            employed_tiles[str(i)] = {'tile': (tile.google[0], tile.google[1], zoom), 'dataset_identifier': file_id}


with open(os.path.join('Gather_tiles', data_name, 'employed_tiles.json'), 'w') as f:
    json.dump(employed_tiles, f)
    

""" Making the mask/label dataset """
from autorasterize import rasterize

for k, obj in employed_tiles.items():
    tile = obj['tile']
    file_id = obj['dataset_identifier']
    google_x, google_y, zoom = tile
    pixel_x_west, pixel_y_north = google_x * TILE_SIZE, google_y * TILE_SIZE
    pixel_x_east, pixel_y_south = (google_x + 1) * TILE_SIZE, (google_y + 1) * TILE_SIZE
    sw = Point.from_pixel(pixel_x_west, pixel_y_south, zoom).meters
    ne = Point.from_pixel(pixel_x_east, pixel_y_north, zoom).meters
    rasterize(polygons_paths[file_id], os.path.join('Gather_tiles', data_name,'label', str(k)+'.png') , sw[0], sw[1], ne[0], ne[1], 1.19)


""" Test and Train Splitter"""
rgb_files = [f for f in os.listdir(rgb_path) if os.path.isfile(os.path.join(rgb_path, f))]
random.shuffle(rgb_files)
files = {}
number_of_all_files = len(rgb_files)
number_of_train_files = int(number_of_all_files*0.7)
files['train'] = rgb_files[:number_of_train_files]
files['test'] = rgb_files[number_of_train_files:]

file_map_path = "./TrainingDataset/" + data_name + "/file_map.json"
with open(file_map_path, 'w') as f:
  print("dumping")
  json.dump(files, f)
    
for k, list_of_files in files.items():
    counter = 0
    print("preparing ", k)
    out_path = os.path.join('./', 'TrainingDataset', data_name, k)
    for file in list_of_files:
        copy2(os.path.join(rgb_path, file), os.path.join(out_path, 'image', str(counter) + '.png'))
        copy2(os.path.join('Gather_tiles', data_name, 'label', file), os.path.join(out_path, 'label', str(counter) + '.png'))
        counter+=1